## Logistic Regression - Titanic

The titanic dataset is a popular dummy dataset used to learn about logistic regression. It has also been used in a [Kaggle data science competition](https://www.kaggle.com/c/titanic), so you'll also find blogposts exploring all kinds of more advanced concepts that use this dataset too! In this assignment, you'll do a logistic regression to look at the effect of sex and class on survival on the titanic, by computing odds ratios.

adapted from: https://github.com/jstray/lede-algorithms/blob/master/week-3/week-3-2-homework.ipynb

Some references:

- [What are odds vs. probability?](https://towcenter.gitbooks.io/curious-journalist-s-guide-to-data/content/analysis/counting_possible_worlds.html)
- [Investigate.ai on Logistic Regressions](https://investigate.ai/regression/logistic-regression-quickstart/)
- [StatQuest Logistic Regressions Playlist](https://www.youtube.com/watch?v=yIYKR4sgzI8&list=PLblh5JKOoLUKxzEP5HA2d-Li7IJkHfXSe)
- [How do I interpret odds ratios in logistic regression?](https://stats.idre.ucla.edu/other/mult-pkg/faq/general/faq-how-do-i-interpret-odds-ratios-in-logistic-regression/) This one's a little more technical, but has good examples.


In [21]:
%load_ext rpy2.ipython
%load_ext autoreload
%autoreload 2

%matplotlib inline  
from matplotlib import rcParams
rcParams['figure.figsize'] = (16, 100)

import warnings
from rpy2.rinterface import RRuntimeWarning
warnings.filterwarnings("ignore") # Ignore all warnings
# warnings.filterwarnings("ignore", category=RRuntimeWarning) # Show some warnings

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, HTML

# show all columns
pd.set_option('display.max_columns', None)

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [22]:
%%javascript
// Disable auto-scrolling
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [23]:
%%R

require('tidyverse')
require('DescTools')

### Load the data

Read in the `titanic.csv` data set again.

In [24]:
# Load titanic.csv
df = pd.read_csv('data/tickets-warnings.csv')
df

,TYPE,CITATION,DATE,DOW,AGENCY,AGENCY2,AGENCY3,LOCAL,OFFICER,LICSTATE,CLASS,CDL,RACE,MINORITY,BLACK,ASIAN,HISPANIC,MIDDLE,NATIVE,SEX,FEMALE,SEARCH,SEARCH2,LOCATE2,LOCATION,TIME,AMPM,TIEMDAY,DAYNIGHT,DAY,DESCRIPT,AMOUNT,MPH,ZONE,MPHOVER,MPHPCT,MPHGROUP,YOB,AGE,AGEGROUP,AGEBAND,ZIP,NEIGHBOR,INNEIGH,REGSTATE,V_MAKE,V_TYPE,V_YEAR,V_AGE,V_AGEGRP,COLOR,HOMESTATE,HOMETOWN,INTOWN,INSTATE,INTOWN2,INSTATE2
0,T,K0001506,20010411,Wednesday,State Police Troop A-4,State Police,S,N,8.247791e+15,MA,D,N,W,W,0.0,0.0,0.0,0.0,0.0,M,0.0,U,NaN,Woburn,Woburn,5,PM,b) afternoon,day,1,SPEEDING,125.0,80,65,15.0,23.0,b) 10 to 15,1980,21.0,21-25,16-25,1876.0,NaN,U,MA,NaN,NaN,0,NaN,U,NaN,MA,Tewksbury,N,Y,0.0,1
1,T,K0001507,20010417,Tuesday,State Police Troop A-4,State Police,S,N,8.247791e+15,MA,D,N,W,W,0.0,0.0,0.0,0.0,0.0,F,1.0,U,NaN,Somerville,Somerville,10,AM,a) morning,day,1,TRAFFIC VIOLATION,50.0,0,0,NaN,NaN,U,1965,36.0,36-40,26-39,2135.0,Allston-Brighton,N,MA,DODG,ARIES,1988,13.0,older,WHITE,MA,Boston,N,Y,0.0,1
2,T,K0001509,20010420,Friday,State Police Troop A-4,State Police,S,N,8.247791e+15,NH,NaN,N,W,W,0.0,0.0,0.0,0.0,0.0,F,1.0,U,NaN,Somerville,Somerville,4,PM,b) afternoon,day,1,FAILURE TO STOP,NaN,0,0,NaN,NaN,U,1940,61.0,61-65,40+,3062.0,NaN,U,NH,NaN,NaN,0,NaN,U,NaN,NH,NaN,N,N,0.0,0
3,T,K0001510,20010420,Friday,State Police Troop A-4,State Police,S,N,8.247791e+15,NH,NaN,N,W,W,0.0,0.0,0.0,0.0,0.0,M,0.0,U,NaN,Woburn,Woburn,8,PM,c) evening,night,0,KEEP IN RIGHT LANE,100.0,0,0,NaN,NaN,U,1949,52.0,51-55,40+,3045.0,NaN,U,NH,NaN,NaN,0,NaN,U,NaN,NH,NaN,N,N,0.0,0
4,T,K0001511,20010427,Friday,State Police Troop A-4,State Police,S,N,8.247791e+15,MA,D,N,W,W,0.0,0.0,0.0,0.0,0.0,M,0.0,U,NaN,Woburn,Woburn,8,AM,a) morning,day,1,SPEEDING,NaN,85,65,20.0,31.0,c) 16 to 20,1977,24.0,21-25,16-25,1835.0,NaN,U,MA,MAZD,B3000,2000,1.0,new,BLACK,MA,Haverhill,N,Y,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
166363,T,K6090282,20010408,Sunday,Gloucester,Gloucester,L,Y,8.247323e+15,MA,D,N,W,W,0.0,0.0,0.0,0.0,0.0,M,0.0,Y,1.0,Gloucester,Gloucester,12,AM,d) predawn,night,0,SPEEDING,170.0,77,55,22.0,40.0,d) more than 20,1977,24.0,21-25,16-25,1930.0,NaN,U,MA,DODG,SPIRIT,1989,12.0,older,WHITE,MA,Gloucester,Y,Y,1.0,1
166364,W,K6228716,20010404,Wednesday,Canton,Canton,L,Y,8.247429e+15,MA,D,N,W,W,0.0,0.0,0.0,0.0,0.0,M,0.0,U,NaN,Canton,Canton,7,AM,a) morning,day,1,SPEEDING,NaN,40,20,20.0,100.0,c) 16 to 20,1966,35.0,31-35,26-39,2021.0,NaN,U,MA,NaN,NaN,0,NaN,U,NaN,MA,Canton,Y,Y,1.0,1
166365,W,K9010748,20010506,Sunday,Weymouth,Weymouth,L,Y,8.247748e+15,MA,D,N,A,M,0.0,1.0,0.0,0.0,0.0,M,0.0,N,0.0,Weymouth,Weymouth,10,AM,a) morning,day,1,SPEEDING,NaN,44,25,19.0,76.0,c) 16 to 20,1926,75.0,71-75,40+,2171.0,NaN,U,MA,NaN,NaN,0,NaN,U,NaN,MA,Quincy,N,Y,0.0,1
166366,W,K9012312,20010525,Friday,State Police Troop D-4,State Police,S,N,8.248466e+15,MA,D,N,W,W,0.0,0.0,0.0,0.0,0.0,M,0.0,U,NaN,Boston Central,Boston,5,PM,b) afternoon,day,1,FAIL TO USE SAFETY,NaN,0,0,NaN,NaN,U,1963,38.0,36-40,26-39,2446.0,NaN,U,MA,BMW,740I,1996,5.0,old,BLUE,MA,Brookline,N,Y,0.0,1


In [25]:
df = df[['TYPE','AGENCY3','SEX','BLACK','ASIAN','HISPANIC','MINORITY','AGE','MPH','MPHOVER','INTOWN', 'INSTATE', 'DAYNIGHT']].copy()
df.head()

,TYPE,AGENCY3,SEX,BLACK,ASIAN,HISPANIC,MINORITY,AGE,MPH,MPHOVER,INTOWN,INSTATE,DAYNIGHT
0,T,S,M,0.0,0.0,0.0,W,21.0,80,15.0,N,Y,day
1,T,S,F,0.0,0.0,0.0,W,36.0,0,NaN,N,Y,day
2,T,S,F,0.0,0.0,0.0,W,61.0,0,NaN,N,N,day
3,T,S,M,0.0,0.0,0.0,W,52.0,0,NaN,N,N,night
4,T,S,M,0.0,0.0,0.0,W,24.0,85,20.0,N,Y,day


### 1. Exploratory data analysis

In [26]:
pd.crosstab(df.MINORITY, df.TYPE)

TYPE,T,W
MINORITY,,
M,16072,12636
U,2306,1439
W,65624,68291


In [27]:
pd.crosstab(df.MINORITY, df.TYPE, normalize='index')

TYPE,T,W
MINORITY,,
M,0.559844,0.440156
U,0.615754,0.384246
W,0.490042,0.509958


In [28]:
df['TICKET'] = df['TYPE'].apply(lambda x: 1 if x == 'T' else 0)

## 2. Logistic Regression

In [44]:
%%R -i df

logistic <- glm(TICKET ~ AGENCY3 + SEX + BLACK + ASIAN + HISPANIC + AGE + MPHOVER + (MPH > 80) + INTOWN + INSTATE + DAYNIGHT , data=df, family="binomial")
print(summary(logistic))
print(PseudoR2(logistic, which = "McFadden"))


Call:
glm(formula = TICKET ~ AGENCY3 + SEX + BLACK + ASIAN + HISPANIC + 
    AGE + MPHOVER + (MPH > 80) + INTOWN + INSTATE + DAYNIGHT, 
    family = "binomial", data = df)

Coefficients:
                Estimate Std. Error z value Pr(>|z|)    
(Intercept)   -1.5263432  0.0502771 -30.359  < 2e-16 ***
AGENCY3L      -0.6362276  0.0305324 -20.838  < 2e-16 ***
AGENCY3O      -0.3350224  0.1765328  -1.898  0.05772 .  
AGENCY3S       0.9260084  0.0336865  27.489  < 2e-16 ***
SEXM           0.2619362  0.0157004  16.683  < 2e-16 ***
SEXU           0.5442921  0.2565028   2.122  0.03384 *  
BLACK          0.1143097  0.0326316   3.503  0.00046 ***
ASIAN          0.2941798  0.0517061   5.689 1.27e-08 ***
HISPANIC       0.5174516  0.0406276  12.736  < 2e-16 ***
AGE           -0.0141756  0.0005764 -24.594  < 2e-16 ***
MPHOVER        0.1692739  0.0020009  84.597  < 2e-16 ***
MPH > 80TRUE   1.7518683  0.0950337  18.434  < 2e-16 ***
INTOWNU       -0.5081336  0.0818181  -6.211 5.28e-10 ***
INTOWNY       

In [45]:
%%R 

exp(coef(logistic))

  (Intercept)      AGENCY3L      AGENCY3O      AGENCY3S          SEXM 
    0.2173289     0.5292854     0.7153221     2.5244126     1.2994436 
         SEXU         BLACK         ASIAN      HISPANIC           AGE 
    1.7233880     1.1210993     1.3420252     1.6777466     0.9859244 
      MPHOVER  MPH > 80TRUE       INTOWNU       INTOWNY      INSTATEY 
    1.1844445     5.7653640     0.6016174     0.7399225     0.7120380 
DAYNIGHTnight 
    0.8824087 
